# GCS Motion Planning - Introduction

This notebook introduces the Graph of Convex Sets (GCS) approach to motion planning.

## What is Graph of Convex Sets (GCS)?

GCS is a powerful optimization-based motion planning approach that:
- Decomposes the configuration space into convex regions
- Creates a graph where nodes are convex sets and edges represent transitions
- Finds optimal trajectories through this graph
- Guarantees collision-free paths within convex regions

**Key Advantages:**
- ✓ Optimal trajectory planning
- ✓ Handles complex obstacles
- ✓ Convex optimization (efficient)
- ✓ Produces smooth, feasible paths

## Setup

In [ ]:
import numpy as np
import sys
sys.path.insert(0, '..')

from src.gcs_planner import GCSBuilder, ConvexSetBuilder, GCSSolver

print("✓ All imports successful")

## Example 1: Simple 2D Planning Problem

In [ ]:
# Create a 2D GCS planner
builder = GCSBuilder(dimension=2)

# Define start region (green box)
start_vertices = np.array([[0, 0], [0.5, 0], [0.5, 0.5], [0, 0.5]])
builder.add_convex_set('start', start_vertices)

# Define middle region
mid_vertices = np.array([[1.5, 0], [2.5, 0], [2.5, 1.0], [1.5, 1.0]])
builder.add_convex_set('middle', mid_vertices)

# Define goal region (red box)
goal_vertices = np.array([[4, 0], [4.5, 0], [4.5, 0.5], [4, 0.5]])
builder.add_convex_set('goal', goal_vertices)

# Connect regions
builder.add_edge('start', 'middle', weight=1.0)
builder.add_edge('middle', 'goal', weight=1.0)

# Get graph info
info = builder.get_graph_info()
print(f"Graph created:")
print(f"  Nodes: {info['num_nodes']}")
print(f"  Edges: {info['num_edges']}")
print(f"  Dimension: {info['dimension']}")

## Example 2: Creating Convex Regions

In [ ]:
# Create different types of convex sets

# Box (hyperrectangle)
box_vertices = ConvexSetBuilder.box(
    center=np.array([0, 0, 0]),
    half_lengths=np.array([1, 1, 1])
)
print(f"Box vertices shape: {box_vertices.shape}")
print(f"Box vertices:\n{box_vertices}")

# Sphere (3D)
sphere_vertices = ConvexSetBuilder.sphere(
    center=np.array([0, 0, 0]),
    radius=1.0,
    num_points=8
)
print(f"\nSphere vertices shape: {sphere_vertices.shape}")

## Example 3: Path Finding

In [ ]:
# Find shortest path through the graph
builder.set_source_target('start', 'goal')
path = builder.get_shortest_path()

if path:
    print(f"Shortest path found: {' -> '.join(path)}")
    print(f"Path length: {len(path)} regions")
else:
    print("No path found!")

## Example 4: Solving for Optimal Trajectory

In [ ]:
# Create solver
solver = GCSSolver(verbose=False, solver='ECOS')

# Define start and goal points
start_point = np.array([0.25, 0.25])
goal_point = np.array([4.25, 0.25])

print(f"Start: {start_point}")
print(f"Goal: {goal_point}")
print(f"\nSolving...")

# Solve (will fail gracefully without full CVXPY setup)
try:
    solution = solver.solve(builder, start_point, goal_point)
    if solution and solution['feasible']:
        print(f"✓ Feasible solution found!")
        print(f"  Waypoints: {solution['length']}")
        print(f"  Cost: {solver.get_trajectory_cost():.4f}")
    else:
        print("✗ No feasible solution found")
except Exception as e:
    print(f"Note: {e}")
    print("This is expected if solvers are not fully configured.")

## Key Concepts Summary

1. **Convex Sets**: Regions where any point between two points in the region stays inside
2. **Graph Nodes**: Each convex region becomes a node
3. **Graph Edges**: Connections between adjacent convex regions
4. **Optimization**: Find trajectory that minimizes cost while staying in convex regions
5. **Guarantees**: Any trajectory found is collision-free (by convexity)

## Next Steps

- Explore `02_visualization_demo.ipynb` for visualization
- Check `03_training_analysis.ipynb` for learning dynamics
- See `04_results_presentation.ipynb` for final results